In [1]:
import pdfplumber
import pandas as pd

# Carrega o arquivo PDF
file_path = "Teste_Python.pdf"

In [3]:
# Ajusta as configurações do pandas para exibir todas as linhas e colunas
pd.set_option('display.max_rows', None)  # Mostrar todas as linhas
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.expand_frame_repr', False)  # Não quebrar a linha do DataFrame

In [ ]:
data = []

# Abre o PDF com pdfplumber
with pdfplumber.open(file_path) as pdf:
    # Itera sobre todas as páginas
    for page in pdf.pages:
        text = page.extract_text().split('\n')
        
        # Filtra as linhas relevantes para as colunas desejadas
        for line in text:
            # Verifica se a linha tem os dados que precisamos. O 0101 evita a leitura de outras linhas que não são Trabalhadores
            if ("Rescis" in line or "Verba Indeniza" in line or "Multa R" in line or "Mensal" in line) and "0101" in line: 
                print(f"Linha lida: {line}")  # Imprime a linha lida que será processada
                
                parts = line.split()
                
                # Comp. Apuração
                comp_apuracao = parts[0]
                
                # Encontra os índices para nome, CPF e total
                indice_inicial = 3
                indice_numerico = 0
                indice_maxalfa = 0
                
                # Percorre o array a partir do índice 3 até encontrar o primeiro elemento não alfanumérico
                for i in range(indice_inicial, len(parts)):
                    if parts[i].isnumeric():
                        indice_numerico = i
                        indice_maxalfa = i - 1
                        break
                
                # Nome Trabalhador (junta partes em uma string)
                nome_trabalhador = " ".join(parts[1:indice_maxalfa + 1])  # +1 para incluir o último elemento alfabético
                
                # CPF e Total (apenas os 11 primeiros números)
                cpf = parts[indice_numerico][:11]  # Pega apenas os 11 primeiros caracteres do CPF
                total = parts[-1]

                # Adiciona os dados extraídos à lista
                data.append([comp_apuracao, nome_trabalhador, cpf, total])
                
                # Imprime os dados extraídos para confirmação
                print(f"Dados extraídos: Comp. Apuração: {comp_apuracao}, Nome: {nome_trabalhador}, CPF: {cpf}, Total: {total}")

# Cria um DataFrame com os dados extraídos
df_extracted = pd.DataFrame(data, columns=["Comp.Apuração", "Nome Trabalhador", "CPF", "Total"])

In [9]:
# Salvar Total como float e não string
df_extracted['Total'] = df_extracted['Total'].str.replace(',', '.').astype(float)

In [19]:
# Salva o DataFrame em um arquivo Excel
output_file = "Trabalhadores_FGTS.xlsx"
df_extracted.to_excel(output_file, index=False)